In [4]:
from __future__ import division
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import average_precision_score
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


#Initialize stop words for Bahasa Indonesia
id_stop_words_file = pd.read_csv('id-stopwords.txt', header=None, names=['stopwords'])
id_stop_words_list = []
for i in range(len(id_stop_words_file)):
    id_stop_words_list.append(id_stop_words_file.values[i][0])

#Initialize Dataset
dataset = pd.read_csv('judul-dan-rangkuman.csv', encoding='latin', header=0, sep=',')
input = dataset['judul_proposal'].str.lower().str.replace('[^a-zA-Z0-9 ]', '')
target = dataset['deskripsi'].str.lower()

#See dataset distribution
print(dataset.deskripsi.value_counts())


#Initialize Stemmer, Stemmernya cukup lama jadi comment aja biar cepet
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()
# for i in range(len(input)):
#    print(i)
#    input[i] = stemmer.stem(input[i])

#Initialize TFIDF Vectorizer
tvect = TfidfVectorizer(min_df=1,stop_words=id_stop_words_list)

#Split Test dan Data Train
x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=4)



tvect1 = TfidfVectorizer(min_df=1,stop_words=id_stop_words_list)
#tvect1 = CountVectorizer()
x_traincv=tvect1.fit_transform(x_train)
x_testcv=tvect1.transform(x_test)
x_traincv=tvect1.fit_transform(x_train)
x_testcv=tvect1.transform(x_test)

#Oversampling data use SMOTE
sm = SMOTE(random_state=4, ratio=1)
# sm = ADASYN()
x_train_res, y_train_res = sm.fit_sample(x_traincv, y_train)

# model = RandomForestClassifier(n_estimators=50) #around 54%
model = LinearSVC() #around 67%
# model = LinearSVC(loss='hinge', multi_class='ovr') # 51%

# y_train = y_train_res.astype('str')
# model.fit(x_traincv,y_train)

y_train_res = y_train_res.astype('str')
model.fit(x_train_res,y_train_res)

print("Cross Validation")
predictions=model.predict(x_testcv)
count = 0
for i in range (len(predictions)):
  if predictions[i]==y_test.values[i]:
     count = count + 1
print(count / len(y_test))

#Evaluation Metrics
print(accuracy_score(y_test, predictions))
print(precision_score(y_test, predictions, average=None))
print(recall_score(y_test, predictions, average=None))  
print(confusion_matrix(y_test, predictions))
print(f1_score(y_test, predictions, average=None))
print(f1_score(y_test, predictions, average='micro'))

print("Same Data")
predictions=model.predict(x_traincv)
count = 0
for i in range (len(predictions)):
  # print(predictions[i])
  # print(y_train.values[i])
  if predictions[i]==y_train.values[i]:
     count = count + 1
print(count / len(y_train))




#SAVE MODEL
# save the model to disk
filename = 'KlasifikasiPakar.sav'
pickle.dump(model, open(filename, 'wb'))

enterprise computing             93
information management           89
intelligent system               84
software engineering             82
sistem komputer                  78
hardware                         40
social and professional issue    40
graphics and visualization       26
algoritma dan pemrograman        14
dasar matematika                  8
Name: deskripsi, dtype: int64


/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


Cross Validation
0.7387387387387387
0.7387387387387387
[0.2        0.         0.80952381 1.         1.         0.59090909
 0.66666667 0.875      0.875      0.73333333]
[0.25       0.         0.80952381 0.5        0.88888889 0.68421053
 0.76923077 0.875      1.         0.61111111]
[[ 1  0  0  0  0  0  0  0  0  3]
 [ 1  0  0  0  0  0  1  0  0  0]
 [ 0  0 17  0  0  3  0  1  0  0]
 [ 0  0  0  1  0  0  0  1  0  0]
 [ 1  0  0  0  8  0  0  0  0  0]
 [ 0  0  3  0  0 13  2  0  0  1]
 [ 0  0  0  0  0  3 10  0  0  0]
 [ 0  0  1  0  0  1  0 14  0  0]
 [ 0  0  0  0  0  0  0  0  7  0]
 [ 2  0  0  0  0  2  2  0  1 11]]
[0.22222222 0.         0.80952381 0.66666667 0.94117647 0.63414634
 0.71428571 0.875      0.93333333 0.66666667]
0.7387387387387387
Same Data
0.9932279909706546


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
from __future__ import division
import pickle
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import average_precision_score
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


#Initialize stop words for Bahasa Indonesia
id_stop_words_file = pd.read_csv('id-stopwords.txt', header=None, names=['stopwords'])
id_stop_words_list = []
for i in range(len(id_stop_words_file)):
    id_stop_words_list.append(id_stop_words_file.values[i][0])

#Initialize Dataset
dataset = pd.read_csv('judul-dan-rangkuman.csv', encoding='latin', header=0, sep=',')
input = dataset['judul_proposal'].str.lower().str.replace('[^a-zA-Z0-9 ]', '')
target = dataset['deskripsi'].str.lower()

#Initialize Stemmer, Stemmernya cukup lama jadi comment aja biar cepet
#factory = StemmerFactory()
#stemmer = factory.create_stemmer()
#for i in range(len(input)):
#    print i
#    input[i] = stemmer.stem(input[i])

#Initialize TFIDF Vectorizer
tvect = TfidfVectorizer(min_df=1,stop_words=id_stop_words_list)

#Split Test dan Data Train
x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=4)

tvect1 = TfidfVectorizer(min_df=1,stop_words=id_stop_words_list)
x_traincv=tvect1.fit_transform(x_train.values.astype('U'))
x_testcv=tvect1.transform(x_test.values.astype('U'))
x_traincv=tvect1.fit_transform(x_train.values.astype('U'))
x_testcv=tvect1.transform(x_test.values.astype('U'))


filename = 'KlasifikasiPakar.sav'
loaded_model = pickle.load(open(filename, 'rb'))

y_train = y_train.astype('str')
loaded_model.fit(x_traincv,y_train)

print("Cross Validation")
predictions=loaded_model.predict(x_testcv)
count = 0
for i in range (len(predictions)):
  if predictions[i]==y_test.values[i]:
     count = count + 1
print(count / len(y_test))


print("Same Data")
predictions=loaded_model.predict(x_traincv)
count = 0
for i in range (len(predictions)):
  if predictions[i]==y_train.values[i]:
     count = count + 1
print(count / len(y_train))

# Contoh 1 data
x = ['aplikasi cloud di kota jakarta']
x_new=tvect1.transform(x)
new_pred = loaded_model.predict(x_new)
print(new_pred)

filename = 'tvect.sav'
pickle.dump(tvect1, open(filename, 'wb'))

Cross Validation
0.4125874125874126
Same Data
1.0
['sistem komputer']
